In [1]:
pip install azureml-monitoring


The following command must be run outside of the IPython shell:

    $ pip install azureml-monitoring

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [1]:
import azureml.core
print(azureml.core.VERSION)

1.0.72


In [2]:
from azureml.core import Workspace
ws = Workspace.create(name='myworkspacemodelmgt',
                      subscription_id='52b56929-ee84-495c-91c3-a84dfacbc9d2',
resource_group='DP100',
create_resource_group=True,
location='centralus'
)

WorkspaceException: WorkspaceException:
	Message: Workspace with name 'myworkspacemodelmgt' already exists under resource group with name 'DP100'.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Workspace with name 'myworkspacemodelmgt' already exists under resource group with name 'DP100'."
    }
}

In [4]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

MBAUER
RG-DP100
eastus
52b56929-ee84-495c-91c3-a84dfacbc9d2


In [5]:
ws.get_details()

{'id': '/subscriptions/52b56929-ee84-495c-91c3-a84dfacbc9d2/resourceGroups/RG-DP100/providers/Microsoft.MachineLearningServices/workspaces/MBAUER',
 'name': 'MBAUER',
 'location': 'eastus',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'tags': {},
 'sku': 'Enterprise',
 'workspaceid': 'b6e69941-a2bc-4089-9e45-3a59f5413063',
 'description': '',
 'friendlyName': '',
 'creationTime': '2020-01-08T12:42:11.5113167+00:00',
 'containerRegistry': '/subscriptions/52b56929-ee84-495c-91c3-a84dfacbc9d2/resourceGroups/RG-DP100/providers/Microsoft.ContainerRegistry/registries/mbauer71590929',
 'keyVault': '/subscriptions/52b56929-ee84-495c-91c3-a84dfacbc9d2/resourcegroups/rg-dp100/providers/microsoft.keyvault/vaults/mbauer0104503571',
 'applicationInsights': '/subscriptions/52b56929-ee84-495c-91c3-a84dfacbc9d2/resourcegroups/rg-dp100/providers/microsoft.insights/components/mbauer1476008790',
 'identityPrincipalId': '9a7704bc-271d-4aa4-8e80-af09ed410df3',
 'identityTenantId': '60623c36-25

# Registering Models to Azure Machine Learning Service

In [6]:
from azureml.core.model import Model
model = Model.register(model_path = "sklearn_regression_model.pkl",
model_name = "sklearn_regression_model.pkl",
tags = {'area': "diabetes", 'type': "regression"},
description = "Ridge regression model to predict diabetes",workspace = ws)

Registering model sklearn_regression_model.pkl


# Querying the Registered Model

In [7]:
regression_models = Model.list(workspace=ws, tags=['area'])
for m in regression_models: print("Name:", m.name,"\tVersion:", m.version, "\tDescription:", m.description, m.tags)

Name: sklearn_regression_model.pkl 	Version: 3 	Description: Ridge regression model to predict diabetes {'area': 'diabetes', 'type': 'regression'}
Name: sklearn_regression_model.pkl 	Version: 2 	Description: Ridge regression model to predict diabetes {'area': 'diabetes', 'type': 'regression'}
Name: sklearn_regression_model 	Version: 1 	Description: Ridge regression model to predict diabetes {'area': 'diabetes', 'type': 'regression'}
Name: sklearn_regression_model.pkl 	Version: 1 	Description: Ridge regression model to predict diabetes {'area': 'diabetes', 'type': 'regression'}


# Register Images Containing Machine Learning Models

In [8]:
from sklearn.externals import joblib
from sklearn.linear_model import Ridge
from azureml.core.model import Model
def init():
    global model
model_path = Model.get_model_path('sklearn_regression_model.pkl')
# deserialize the model file back into a sklearn model
model = joblib.load(model_path)
# note you can pass in multiple rows for scoring
def run(raw_data):
    try:
        data = json.loads(raw_data)['data']
        data = numpy.array(data)
        result = model.predict(data)
# you can return any datatype if it is JSON-serializable
        return result.tolist()
    except Exception as e:
       error = str(e)
    return error

In [9]:
from azureml.core.conda_dependencies import CondaDependencies
myenv = CondaDependencies.create(conda_packages=['numpy','scikit-learn'])
with open("myenv.yml","w") as f: f.write(myenv.serialize_to_string())

In [13]:
from azureml.core.image import Image, ContainerImage
image_config = ContainerImage.image_configuration(runtime= "python",
execution_script="score.py",
conda_file="myenv.yml",
tags = {'area': "diabetes", 'type': "regression"},
description = "Image with ridge regression model")
image = Image.create(name = "myimage1",
# this is the model object
models = [model],image_config = image_config,workspace = ws)
image.wait_for_creation(show_output = True)

ERROR - Models must either be of type azureml.core.Model or a str path to a file or folder.



WebserviceException: WebserviceException:
	Message: Models must either be of type azureml.core.Model or a str path to a file or folder.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Models must either be of type azureml.core.Model or a str path to a file or folder."
    }
}